# Transcript Summarization
This notebook reads the transcript dataset and generates summaries using the Bert Extractive Summarizer.

There are x methods of Bert Summarization
* Bert Extractive Summarizer
* SBert Summarizer

Hyperparameters to be considered for models
* param ratio: Ratio of sentences to use.
* param min_length: Minimum length of sentence candidates to utilize for the summary.
* param max_length: Maximum length of sentence candidates to utilize for the summary.
* param use_first: Whether or not to use the first sentence.
* param algorithm: Which clustering algorithm to use. (kmeans, gmm)
* param num_sentences: Number of sentences to use (overrides ratio).
* param return_as_list: Whether or not to return sentences as list.

In [136]:
import pandas as pd
import numpy as np
from summarizer import Summarizer
from summarizer.sbert import SBertSummarizer


In [137]:
import ttg


In [143]:
import re

In [154]:

email = #lines[1]
m = re.search("[0-9][0-9]:[0-9][0-9]", email)
email[:m.start()] + email[m.end():]

SyntaxError: invalid syntax (<ipython-input-154-f5d8fbc521aa>, line 1)

In [157]:
#m = re.sub("[0-9][0-9]:[0-9][0-9]", '', lines[1])
#m

In [235]:
with open('data/demo.txt') as f:
    lines = f.readlines()
    lines = ''.join(lines)

In [236]:
lines = re.sub("[0-9][0-9]:[0-9][0-9]", '', lines)
lines = re.sub("[0-9]:[0-9][0-9]", '', lines)
lines = re.sub("\'", '', lines)
lines = re.sub("\n", '', lines)

In [250]:
#lines
with open('data/siads697698/01_week-1/02_videos/05_how-to-do-a-standup.en.txt') as f:
    lines = f.readlines()
    lines = ''.join(lines)
lines = re.sub("\n", ' ', lines)

In [251]:
lines

"I mentioned to you that we're going to do some biweekly stand-ups, so here's what to do in a stand-up. Some of you who might have worked at tech companies or similar organizations might already know about them, but if you're new, here is roughly what we're going to do. When it comes time for your stand-up, you're going to get on your webcam, or your screen recording tool, doesn't matter.Z You can show your face or not, and you're going to answer the following questions. What did my team work on this past week? What are we working on now? What issues are blocking us? Even if you're not especially blocked by anything, just say what are some of the challenges that you're facing or things that you're not sure of. To emphasize here, any team member can make the recording, so it doesn't have to be everybody on the team. I just want one representative from the team to make the stand-up and feel free to use your screen-sharing to show us any code snippets or cool results. It can be quite casu

# Read in Transcript DataFrame

In [4]:
df = pd.read_csv("./data/transcripts", index_col=0)
df.head()

,course_numer,transcripts,length
0,630,Hi and welcome to the Master of Applied Data S...,3079
1,630,"All right. So, I got a question in the forum f...",9387
2,630,"Hi, Daniel. Hello, Cohan. Hi. How are you? Ver...",20115
3,630,"the red line. Now, next time we draw another 5...",20116
4,630,"In this video, we're going to take a closer lo...",16163


## Generate First Transcript to Summarize


In [252]:
#body = df.transcripts.iloc[6] #tried = [0, 5 
#print(len(body.split('. ')))
#print(body)

body = lines

## Bert Extractive Summarizer

### Initialize Summarizer

In [66]:
?model

Signature:      
model(
    body: str,
    ratio: float = 0.2,
    min_length: int = 40,
    max_length: int = 600,
    use_first: bool = True,
    algorithm: str = 'kmeans',
    num_sentences: int = None,
    return_as_list: bool = False,
) -> str
Type:            Summarizer
String form:     <summarizer.bert.Summarizer object at 0x7fbe25981fd0>
File:            ~/opt/anaconda3/lib/python3.7/site-packages/summarizer/bert.py
Docstring:       <no docstring>
Class docstring: Summarizer based on the BERT model.
Init docstring: 
This is the main Bert Summarizer class.

:param model: This parameter is associated with the inherit string parameters from the transformers library.
:param custom_model: If you have a pre-trained model, you can add the model class here.
:param custom_tokenizer: If you have a custom tokenizer, you can add the tokenizer here.
:param hidden: This signifies which layer of the BERT model you would like to use as embeddings.
:param reduce_option: Given the output of the 

In [241]:
#model = Summarizer(random_state=42)
model = Summarizer('distilbert-base-uncased', hidden=[-2], hidden_concat=True, random_state=42)
result = model(body, num_sentences=5, use_first=True)

#hidden layer goes [-7, 6]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [242]:
result

'Im the managing director at the Michigan Institute for data science. And in any team sport, its really hard to evaluate defense, especially as I mentioned in this previous video,  the traditional statistics that attract for defense are steals and blocks. Another, work thats been done with this data, and you could see how basketball is borrowing methods  from other fields is dividing this, the court reality to the player. And so you can see, how good a players at positioning themselves on defense,  on your own, the offense from looking at this kind of analysis  Next, so this is something that doesnt use the player tracking data uses video data. - Ivana do you want to answer another few questions.'

In [130]:
strs

['-7', '-6', '-5', '-4', '-3', '-2', '-1', '0', '1', '2', '3', '4', '5', '6']

In [108]:
# Short and simple approach
def permute(list1):
    superSet = []
    n = len(list1)

    for i in range(2**n, 2**(n+1)):
        seq = [x for x in bin(i)[3:]]
        superSet.append(''.join([list1[j]
                                 for j in range(n) if seq[j] == '1']))

    return len(superSet)


print(permute(strs))
# Output : ['', 'C', 'B', 'BC', 'A', 'AC', 'AB', 'ABC']

16384


In [124]:
import itertools

In [126]:
c = list(itertools.product(strs, strs))
unq = set(c)

In [128]:
len(unq)

196

### Generate Result

In [47]:
#result = model(body, num_sentences=6, use_first=False) #min_length=60

In [48]:
#?model

### Print Result

In [72]:
full = ''.join(result)
print(full)

This module is about randomized experiments. As mentioned on the previous slide, some people did not receive the treatment to which they were assigned to, so there is a problem of noncompliance. The first outcome variable that we consider is the use of emergency departments.


This module is about randomized experiments. Now, this is easy to determine because of random assignment, we can just compare averages across treatment and control groups. Those who were offered to enroll in Medicaid went about 0.1 times more often to the ER as shown in Column 4.

## SBert Summarizer

### Initialize Summarizer

In [73]:
model_s = SBertSummarizer('paraphrase-MiniLM-L6-v2', random_state=42)

In [74]:
?model_s

Signature:     
model_s(
    body: str,
    ratio: float = 0.2,
    min_length: int = 40,
    max_length: int = 600,
    use_first: bool = True,
    algorithm: str = 'kmeans',
    num_sentences: int = None,
    return_as_list: bool = False,
) -> str
Type:           SBertSummarizer
String form:    <summarizer.sbert.SBertSummarizer object at 0x7fbe681cb050>
File:           ~/opt/anaconda3/lib/python3.7/site-packages/summarizer/sbert.py
Docstring:     
The SBert Summarizer.

This is based on the Sentence Bert Summarizer.
Init docstring:
SBert Summarizer.

:param model: The model for the sentence transformer.
:sentence_handler: The handler to process sentences. If want to use coreference, instantiate and pass.
:param random_state: The random state to reproduce summarizations.
Call docstring:
(utility that wraps around the run function)
Preprocesses the sentences, runs the clusters to find the centroids, then combines the sentences.

:param body: The raw string body to process.
:param ratio

### Generate Result

In [11]:
result = model(body, num_sentences=6)

### Print Result

In [12]:
full = ''.join(result)
print(full)

Hi and welcome to the Master of Applied Data Science Course and Causal Inference. For example, if you hit the snooze button on your alarm clock, then the alarm will stop. But sometimes actions are more complex, or we have to wait longer to observe the outcome. The problem is, that we cannot always run experiments. So, we often have to work with observational data, that is, data where we do not have full control over whether people take a certain action or not. We will talk about the core ideas behind those methods, their underlying assumptions, and what kind of causal effect they're estimating?


## Try Out Different Combinations

In [253]:
#dictionary of results
Tracker = {}

#params

param_use_first = [True, False]
param_algorithm = ['kmeans', 'gmm']
param_num_sentences = [3, 5, 8]

#models
model_bert = Summarizer(random_state=42)
model_sbert = SBertSummarizer('paraphrase-MiniLM-L6-v2', random_state=42)

for u in param_use_first:
    for a in param_algorithm:
        for n in param_num_sentences:
            result_bert = model_bert(body, num_sentences=n, use_first=u, algorithm=a) #min_length=60
            full_bert = ''.join(result_bert)
            result_sbert = model_sbert(body, num_sentences=n, use_first=u, algorithm=a)
            full_sbert = ''.join(result_sbert)
            Tracker['bert {} {} {}'.format(u, a, n)] = full_bert
            Tracker['sbert {} {} {}'.format(u, a, n)] = full_sbert
            

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [254]:
for x, i in Tracker.items():
    print('\n\t***{}***\n'.format(x))
    print(i)


	***bert True kmeans 3***

I mentioned to you that we're going to do some biweekly stand-ups, so here's what to do in a stand-up. Finally, let me tell you how to share your stand-up. Then I will be asking you to watch and comment on two other classmates stand-ups, one stand up per team every two weeks, don't worry, I will remind you when they are coming, you will not be in the dark about this, and I can also create, you know, I'll put one in the channel, a demo, so you can see a real one.

	***sbert True kmeans 3***

I mentioned to you that we're going to do some biweekly stand-ups, so here's what to do in a stand-up. It doesn't really matter as long as it's somewhere that you can upload the video and share a link to it. Then I will be asking you to watch and comment on two other classmates stand-ups, one stand up per team every two weeks, don't worry, I will remind you when they are coming, you will not be in the dark about this, and I can also create, you know, I'll put one in the c

## Try Out Different Hidden Layers

In [134]:
#dictionary of results
Tracker_hidden = {}

#params

param_hidden =  list(range(-7, 7))
param_algorithm = 'gmm'
param_num_sentences = 5
param_use_first = True

#models



for h in param_hidden:
    model_bert = Summarizer(random_state=42, hidden = h)
    result_bert = model_bert(body, num_sentences=param_num_sentences, use_first=param_use_first, algorithm=param_algorithm) #min_length=60
    full_bert = ''.join(result_bert)
    Tracker['bert {} {} {}'.format(h, param_use_first, param_algorithm, param_num_sentences)] = full_bert         

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.dec

In [135]:
for x, i in Tracker_hidden.items():
    print('\n\t***{}***\n'.format(x))
    print(i)

In [2]:
# column names
feature_col = "features" # feature vector
value_col = "foldClass" # fold class to be predicted

In [3]:
#df = pd.read_json("./intermediate_data/foldClassification.json")

## Create a Word2vec Model
We use the **ProtVec model** (Asgari et al.) to calculate a 100-dimensional feature vector for each protein sequence. ProtVec uses a Word2vec model (Mikolov et al.) that has been trained on 546,790 sequences in [Swiss-Prot](https://web.expasy.org/docs/swiss-prot_guideline.html) using 546,790 × 3 = 1,640,370 sequences of 3-grams. The 3-grams represent "biological words" in a protein sequence, e.g., sequence: SRMPSPP -> 3-grams: SRM RMP MPS PSP SPP. The **ProtVec** model is available for download at: https://github.com/ehsanasgari/Deep-Proteomics.

Asgari E, Mofrad MR (2015) Continuous Distributed Representation of Biological Sequences for Deep Proteomics and Genomics, PLoS One. 10(11):e0141287. doi: [10.1371/journal.pone.0141287](https://doi.org/10.1371/journal.pone.0141287).

Mikolov T, Sutskever I, Chen K, Corrado GS, Dean J, Distributed representations of words and phrases and their compositionality. In: [Advances in neural information processing systems; 2013. p. 3111–3119.](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

#### Read ProtVec Model
Next we read a local copy of the ProtVec model. The ProtVec model is represented as a dictionary, with the 3-gram as the key and the 100-dimensional feature vector as the value.

In [4]:
protvec = protvectors.read_protvectors("./data/protVec_100d_3grams.csv")

print("Example ProtVec for 3-gram SRM:\n", protvec['SRM'])

Example ProtVec for 3-gram SRM:
 [-0.349053 -0.034172 -0.14602  -0.112906  0.318846  0.100117 -0.104718
 -0.194695 -0.08249   0.016351 -0.181182  0.109543  0.067238 -0.027135
  0.222703  0.073312 -0.074177 -0.087137 -0.27853   0.003309 -0.065516
 -0.035587  0.042179  0.169955  0.155156 -0.07882   0.203758  0.129488
 -0.009507 -0.033186 -0.007172 -0.039388  0.243934  0.009303  0.043914
 -0.018962 -0.23077  -0.136273  0.027782  0.232346 -0.2341    0.102889
 -0.054253 -0.111376  0.106518 -0.027139 -0.139712 -0.049569  0.057983
 -0.157097  0.090227  0.0228    0.114038  0.017181 -0.015422 -0.035576
 -0.014446  0.000584 -0.292332  0.003074  0.097327  0.072325  0.138753
  0.028772 -0.023035  0.024519  0.123589  0.021453  0.286168  0.094651
 -0.145597  0.132008 -0.104951  0.121934 -0.042467 -0.075287  0.306096
  0.096278 -0.121827  0.167771  0.059359 -0.169576  0.018486 -0.143597
  0.211764  0.171916  0.200995  0.190091 -0.142053  0.022641  0.204606
 -0.083642  0.016121 -0.147855  0.001436 -0.

## Create 3-grams of the Protein Sequence
Next, we create 3-grams for the protein sequences in our dataset.

In [5]:
# add column ngram to dataframe
df['ngram'] = df.sequence.apply(protvectors.ngrammer, n=3)
df.head(3)

,Exptl.,FreeRvalue,R-factor,alpha,beta,coil,foldClass,length,pdbChainId,resolution,secondary_structure,sequence,ngram
1,XRAY,0.26,0.19,0.469945,0.046448,0.483607,alpha,366,16VP.A,2.100,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,"[SRM, RMP, MPS, PSP, SPP, PPM, PMP, MPV, PVP, ..."
1000,XRAY,0.23,0.18,0.504630,0.004630,0.490741,alpha,216,1PBW.B,2.000,CCCCCCCCCCCCCCHHHHCCTTSCSCHHHHHHHHHHHHHHTTCTTT...,MEADVEQQALTLPDLAEQFAPPDIAPPLLIKLVEAIEKKGLECSTL...,"[MEA, EAD, ADV, DVE, VEQ, EQQ, QQA, QAL, ALT, ..."
10002,XRAY,0.26,0.22,0.716172,0.006601,0.277228,alpha,303,4TQ3.A,2.408,CCCCCCCCCCCCCCCHHHHHHCGGGGHHHHHHHHHHHHHHCCTTSC...,MDSSLANINQIDVPSKYLRLLRPVAWLCFLLPYAVGFGFGITPNAS...,"[MDS, DSS, SSL, SLA, LAN, ANI, NIN, INQ, NQI, ..."


## Create a Fixed-sized Feature Vector
Here we create a 100-dimensional feature vector by adding up the ProtVectors for all 3-grams in a protein sequence and standardize each feature vector to zero-mean and unit-variance. 

In [6]:
df[feature_col] = df.ngram.apply(protvectors.apply_protvectors, protvec=protvec)

df.head(3)

,Exptl.,FreeRvalue,R-factor,alpha,beta,coil,foldClass,length,pdbChainId,resolution,secondary_structure,sequence,ngram,features
1,XRAY,0.26,0.19,0.469945,0.046448,0.483607,alpha,366,16VP.A,2.100,CCSCCCCCCCCHHHHHHHHHHHHTCTTHHHHHHHHHHCCCCCSTTS...,SRMPSPPMPVPPAALFNRLLDDLGFSAGPALCTMLDTWNEDLFSAL...,"[SRM, RMP, MPS, PSP, SPP, PPM, PMP, MPV, PVP, ...","[-2.618341208445193, -0.37215537192569575, 0.1..."
1000,XRAY,0.23,0.18,0.504630,0.004630,0.490741,alpha,216,1PBW.B,2.000,CCCCCCCCCCCCCCHHHHCCTTSCSCHHHHHHHHHHHHHHTTCTTT...,MEADVEQQALTLPDLAEQFAPPDIAPPLLIKLVEAIEKKGLECSTL...,"[MEA, EAD, ADV, DVE, VEQ, EQQ, QQA, QAL, ALT, ...","[-2.4130836608297224, -0.5122827315971855, 0.1..."
10002,XRAY,0.26,0.22,0.716172,0.006601,0.277228,alpha,303,4TQ3.A,2.408,CCCCCCCCCCCCCCCHHHHHHCGGGGHHHHHHHHHHHHHHCCTTSC...,MDSSLANINQIDVPSKYLRLLRPVAWLCFLLPYAVGFGFGITPNAS...,"[MDS, DSS, SSL, SLA, LAN, ANI, NIN, INQ, NQI, ...","[-2.6375752438981404, 0.18385725798670652, 0.2..."


## Save DataFrame with Feature Vectors
We save the dataset with protein sequence, fold classification, and feature vectors as a Pandas dataframe for further analysis.

In [7]:
df.to_json("./intermediate_data/features.json")

## Next step
After you saved the dataset here, run the next step in the workflow [3-FitModel.ipynb](./3-FitModel.ipynb) or go back go back to [0-Workflow.ipynb](./0-Workflow.ipynb).

---

**Authors:** [Peter W. Rose](mailto:pwrose.ucsd@gmail.com), Shih-Cheng Huang, UC San Diego, October 1, 2018

---